# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
import time
import pickle
from itertools import count
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
# Using the 20-agent version:
env = UnityEnvironment(file_name='../Reacher_Windows_x86_64_20/Reacher.exe')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
random_actions = False

In [ ]:
if random_actions:
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
if random_actions:
    env.close()

### 4. Training

When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
state_size = states.shape[1]

In [ ]:
def ddpg(name, n_max_episodes=500):
    scores_global = []                                    # collect scores for each episode
    scores_deque = deque(maxlen=100)                      # collect 100 latest scores
    start_learning = time.time()

    for i_episode in range(1, n_max_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment    
        states = env_info.vector_observations             # get the current state (for each agent)
        scores = np.zeros(num_agents)                     # initialize the score (for each agent)
        
        agent.reset()                                     # reset noise process
        start_episode = time.time()
        for _ in count():
            actions = agent.act(states)
            
            env_info = env.step(actions)[brain_name]      # send all actions to the environment
            next_states = env_info.vector_observations    # get next state (for each agent)
            rewards = env_info.rewards                    # get reward (for each agent)
            dones = env_info.local_done                   # see if episode finished
            scores += env_info.rewards                    # update the score (for each agent)
            
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done)  # save experience [and learn]
            
            if np.any(dones):                             # exit loop if episode finished
                break
            
            states = next_states                          # roll over states to next time step
        
        score = np.mean(scores)                           # average over 20 agents for this episode
        scores_deque.append(score)
        scores_global.append(score)
        
        print('{}-th episode, mean[100]= {:.2f}, mean[ep]= {:.2f}, max[ep]= {:.2f}, min[ep]= {:.2f}, std[ep]= {:.2f}, duration[ep]= {:.2f}'\
              .format(i_episode, np.mean(scores_deque), score, np.max(scores), np.min(scores), np.std(scores), time.time()-start_episode), end="\n")
        
        if i_episode % 10 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        
        if np.mean(scores_deque) > 10:
            print("SUCCESS -- after {} episodes and {:.2f} min".format(i_episode, (time.time()-start_learning)/60))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor'+str(name)+'success.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic'+str(name)+'success.pth')
            
            return scores_global
            
    return scores_global

In [ ]:
# loop mechanism to compare seeds
# for i in range(1, 100):
# for i in [3, 15, 25, 33, 36]:
for i in [33]:
    print("--- seed = {} ---".format(i))
    agent = Agent(state_size=state_size, action_size=action_size, random_seed=i)
    scores = ddpg(name=i)

    print(scores)
    # save scores
    logfile_name = str(i)+".txt"
    with open(logfile_name, "wb") as fp:
        pickle.dump(scores, fp)

### 5. Testing

In [ ]:
agent = Agent(state_size=state_size, action_size=action_size, random_seed=36)
agent.actor_local.load_state_dict(torch.load('../src_submission/checkpoint_actor36.pth'))
agent.critic_local.load_state_dict(torch.load('../src_submission/checkpoint_critic36.pth'))

In [ ]:
for i_episode in range(10):
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    agent.reset()
    for i_step in range(1000):
        actions = agent.act(states)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations
        states = next_states
        if np.any(env_info.local_done):                                  # exit loop if episode finished
                break

In [ ]:
env.close()